In [1]:
from typing import Any
from langchain import LLMChain

import openai
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain


In [2]:
"""Chain of Density prompting: https://arxiv.org/abs/2309.04269"""
user_sum_prompt = """
Article: {text}
You will generate increasingly concise entity-dense summaries of the above article. Repeat the following 2 steps 5 times.

Step 1: Identify 1-3 informative entities (delimited) from the article which are missing from the previously generated summary.
Step 2: Write a new denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities.

A missing entity is
- Relevant: to the main stories.
- Specific: descriptive yet concise (5 words or fewer).
- Novel: not in the previous summary.
- Faithful: present in the article.
- Anywhere: located in the article.

Guidelines:
- The first summary should be long (4-5 sentences, ~80 words), yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.
- Make every word count. Rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise, yet self-contained, e.g., easily understood without the article.
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities.

Remember: Use the exact same number of words for each summary.

Do not include any explanations. Only return a valid valid RFC8259 compilant JSON.
The JSON should be an array of length 5 following this format:
{{
"data": [
   {{
      "missing_entities":  "An array of missing entitys"
      "denser_summary": "denser summary, covers every entity in detail"
   }}
]
}}
The response in JSON format:
"""

user_translate_prompt = """
Übersetze das folgende JSON in {language}. Beinhalte die Formatierung als RFC8259 JSON bei.
Das JSON sollte ein Array der Länge 5 sein, welcher folgendem Format folgt:
{{
"data": [
   {{
      "missing_entities": "An array of missing entitys"
      "denser_summary": "denser summary, covers every entity in detail"
   }}
]
}}

JSON: {sum}
"""


In [3]:
def getSummarizationPrompt() -> PromptTemplate:
    return PromptTemplate(input_variables=["text"], template=user_sum_prompt)

def getTranslationPrompt() -> PromptTemplate:
    return PromptTemplate(input_variables=["language", "sum"], template=user_translate_prompt)


In [4]:
import os
os.environ['http_proxy'] = "http://internet-proxy-client.muenchen.de:80" 
os.environ['https_proxy'] = "http://internet-proxy-client.muenchen.de:80" 

In [5]:
openai.api_type = "azure"
openai.api_base = f"https://cog-tb726faeoukc2.openai.azure.com"
openai.api_version = "2023-05-15"
openai.api_key = "9324d0a0d0bd4d72b0e34596a9dc4a9c"

In [6]:
from typing import Dict
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema.output import LLMResult
class MyCustomHandler(BaseCallbackHandler):
    def on_text(self, text: str, **kwargs: Any) -> Any:
        print(f"Text: {text}")
        self.log = text

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """Run when LLM ends running."""
        print(response.llm_output.keys())
        print(response.llm_output)

In [7]:
verbose = False
llm = AzureChatOpenAI(
    model="gpt-35-turbo",
    temperature= 0.7,
    max_tokens=1000,
    n=1,
    deployment_name= "chat",
    openai_api_key=openai.api_key,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    openai_api_type=openai.api_type,
    callbacks=[MyCustomHandler()]
)
summarizationChain = LLMChain(llm=llm, prompt=getSummarizationPrompt(), output_key="sum", verbose=verbose)
translationChain = LLMChain(llm=llm, prompt=getTranslationPrompt(), output_key="translation", verbose=verbose)
overall_chain = SequentialChain(
    chains=[summarizationChain, translationChain], 
    input_variables=["language", "text"],
    output_variables=["translation", "sum"],
    verbose=verbose
)

In [8]:
text ="""Der Zoo Hof vermisst seit dem Wochenende das Känguru Willi. Das Tier sei seitdem mehrmals gesichtet worden, sagte ein Polizeisprecher am Dienstag. "Es ist bisher nicht gelungen, es einzufangen." Mehrere Medien hatten zuvor darüber berichtet.

Der Zoo schrieb auf Instagram, dass das graubraune Känguru-Männchen Willi am Samstagmorgen nicht mehr im Gehege gewesen sei. Wie das etwa 70 Zentimeter große Tier den Zoo verlassen konnte, sei noch in Klärung. "Da das Tier von Blättern und Gras lebt, befindet es sich momentan nicht in der Gefahr des Verhungerns, einzig die Beutegreifer in der Natur könnten ihm gefährlich werden. Wer das Känguru sehe, soll es nur aus der Ferne beobachten und die Polizei verständigen, bat der Zoo im bayerischen Oberfranken. "Bitte unterlassen Sie Suchen auf eigene Faust, dies würde das Tier nur noch weiter unnötig stressen."

Entlaufene Kängurus wurden in den vergangenen Jahren immer mal wieder in Bayern gemeldet. Zuletzt war im Sommer das Känguru-Männchen Toni aus dem Gehege eines Hotels im Bayerischen Wald ausgebüxt. Mit einer Spur aus Leckereien konnten die Besitzer es nach mehreren Wochen schließlich in sein Gehege zurück locken."""
text ="""Autonome Autos sind Autos, die komplett selbstständig fahren. Sie brauchen keinen Fahrer mehr. Ihr Innenraum könnte beispielsweise wie in einem Zug aussehen, mit vier Sitzen und eventuell einem Tisch. Lenkrad, Pedale, Schaltknüppel – all das wäre nicht mehr vorhanden. Damit diese Autos autonom fahren können, sind sie mit einer Fülle von Technik ausgestattet. Laser, Sensoren, Kameras, ein GPS-Empfänger, Messgeräte und ein Bordcomputer erkennen andere Verkehrsteilnehmer, Straßenschilder, Ampeln usw. Sie machen es möglich, dass das Auto sich sicher durch den Verkehr bewegt und Kollisionen vermeidet. Mehrere Autohersteller arbeiten an verschiedenen Modellen autonomer Autos und die Forschung ist schon weit fortgeschritten."""
text ="""

Viele wollen es. 57 Prozent der Deutschen sagen in einer Umfrage des Meinungsforschungsinstituts YouGov, dass sie für ein Tempolimit von 130 Stundenkilometern auf Autobahnen sind. Genauso sind es Menschen aus Wissenschaft, Politik, Kirche und dem öffentlichen Leben, die im Bündnis Tempolimit jetzt! aktiv sind. Einer davon: Sebastian Vettel, der vier Saisons der Formel-1 gewonnen hat.​

Einer, der es definitiv nicht will, ist Verkehrsminister Volker Wissing. Der Politiker aus der Freien Demokratischen Partei (FDP) ist gegen staatliche Limitierungen im Straßenverkehr. Er sagt: „Autofahren bedeutet Freiheit.“ Mit dieser Meinung ist Wissing in seiner Partei und in Deutschland nicht allein.​

Fast allein ist das Land aber mit der Möglichkeit, auf Autobahnen so schnell zu fahren, wie man will. In Europa hat nur ein anderer Ort kein Limit: die Isle of Man in der Irischen See. Autobahnen gibt es dort aber keine. Auch insgesamt gibt es nur wenige andere Länder ohne Maximalgeschwindigkeit, sie haben aber schlechte Straßen. ​

Über ein Tempolimit diskutiert man in Deutschland schon ziemlich lang – und meistens emotional. Dabei ist 1953 ein wichtiges Jahr. Damals fand unter Kanzler Konrad Adenauer eine große Deregulierung der Geschwindigkeit statt: Menschen in Pkw und auf Motorrädern durften überall in der Bundesrepublik so schnell fahren, wie sie wollten und konnten. Westdeutschland lebte seinen Mythos der Autobahn. Anders in der Deutschen Demokratischen Republik: Dort waren 100 Stundenkilometer auf Autobahnen das Maximum, 80 außerhalb von Orten und 50 in Orten.​

Weil in der Bundesrepublik die Zahl der Verkehrstoten deutlich stieg, führte die Regierung im Jahr 1957 für Orte eine Maximalgeschwindigkeit von 50 Stundenkilometern ein. Diese Norm hatte viele Gegner: den Allgemeinen Deutschen Automobil-Club (ADAC), die Autoindustrie, aber auch Menschen aus Politik und Wissenschaft.​

Im Jahr 1972 kam die Einführung eines Tempolimits von 100 Stundenkilometern auf Landstraßen. Auch über diese Aktion gegen die vielen Verkehrstoten gab es intensive Diskussionen. ​

Monate später kam die Ölkrise – und mit ihr ab November 1973 ein vorübergehendes Tempolimit von 100 Stundenkilometern auf Autobahnen. So wollte man den Erdölverbrauch reduzieren. Aber der Protest war groß. Ganz vorn dabei war der ADAC, der eine Kampagne mit dem Motto „Freie Fahrt für freie Bürger“ startete. Mit Erfolg: Nach 111 Tagen endete das Tempolimit auf Autobahnen. Ab dem 15. März 1974 durfte man wieder so viel Gas geben, wie man wollte. Das ist bis heute auf circa 70 Prozent der deutschen Autobahnen möglich. Dort gibt es nur die Empfehlung für eine Geschwindigkeit von 130 Stundenkilometern.  ​

Die Debatte über das Tempolimit hat trotzdem nicht aufgehört. Und auch 50 Jahre später spielen die mächtige deutsche Autolobby und das (oft auch von ihr verwendete) Freiheitsargument noch immer zentrale Rollen – obwohl die Welt heute eine andere ist.​

Da ist vor allem die Frage, ob ein Tempolimit gegen die Klimakrise hilft. Anfang dieses Jahres hat das Umweltbundesamt eine Studie publiziert. Sie zeigt, dass ein Limit von 120 Stundenkilometern einen größeren Effekt hätte als bis jetzt gedacht. Pro Jahr könnte man die Treibhausgasemissionen des deutschen Straßenverkehrs so um 4,2 Prozent reduzieren.​

Elementar ist außerdem die Frage, ob ein Tempolimit den Verkehr sicherer macht. Klar ist: Je schneller ein Auto unterwegs ist, desto länger dauert das Bremsen bis zum Stoppen. Das Nachrichtenportal Spiegel Online hat den Unfallatlas der statistischen Ämter des Bundes und der Länder analysiert. Das Resultat: Je eine Milliarde gefahrener Kilometer gibt es auf Autobahnabschnitten mit Tempolimit 0,95 tödliche Unfälle. Auf Strecken ohne Tempolimit sind es 1,67 tödliche Unfälle – rund 75 Prozent mehr.​

Warum lieben viele Deutsche das schnelle Fahren trotzdem so sehr? Fragt man Verkehrspsychologen, hört man bald wieder das große Wort dieser Debatte: Freiheit. Aber dominiert der Wunsch danach wirklich die kollektive Psyche und entscheidet über das Tempo deutscher Autofahrerinnen?​

Eine Antwort liefert eine Studie des Instituts der deutschen Wirtschaft aus dem Jahr 2021. Sie zeigt: 77 Prozent der Autofahrer sind auf Autobahnabschnitten ohne Tempolimit langsamer als 130 Stundenkilometer unterwegs. Zwölf Prozent fahren dort zwischen 130 und 140 Stundenkilometer. Nur weniger als zwei Prozent fahren schneller als 160.​

Inzwischen ist auch die Mehrheit der Mitglieder des ADAC für ein Tempolimit. Der Verband der Automobilindustrie ist noch dagegen. Und die Regierung? Im Koalitionsvertrag steht auf Wunsch der FDP: „Ein generelles Tempolimit wird es nicht geben.“
"""

In [9]:
language="deutsch"

In [10]:

from langchain.callbacks import get_openai_callback
with get_openai_callback() as cb:
    result = overall_chain({"text": text, "language": language})
total_tokens = cb.total_tokens

dict_keys(['token_usage', 'model_name'])
{'token_usage': <OpenAIObject at 0x29bfe740cc0> JSON: {
  "prompt_tokens": 1667,
  "completion_tokens": 684,
  "total_tokens": 2351
}, 'model_name': 'gpt-35-turbo'}
dict_keys(['token_usage', 'model_name'])
{'token_usage': <OpenAIObject at 0x29bfe73e8b0> JSON: {
  "prompt_tokens": 781,
  "completion_tokens": 958,
  "total_tokens": 1739
}, 'model_name': 'gpt-35-turbo'}


In [11]:
result, total_tokens

({'text': '\n\nViele wollen es. 57 Prozent der Deutschen sagen in einer Umfrage des Meinungsforschungsinstituts YouGov, dass sie für ein Tempolimit von 130 Stundenkilometern auf Autobahnen sind. Genauso sind es Menschen aus Wissenschaft, Politik, Kirche und dem öffentlichen Leben, die im Bündnis Tempolimit jetzt! aktiv sind. Einer davon: Sebastian Vettel, der vier Saisons der Formel-1 gewonnen hat.\u200b\n\nEiner, der es definitiv nicht will, ist Verkehrsminister Volker Wissing. Der Politiker aus der Freien Demokratischen Partei (FDP) ist gegen staatliche Limitierungen im Straßenverkehr. Er sagt: „Autofahren bedeutet Freiheit.“ Mit dieser Meinung ist Wissing in seiner Partei und in Deutschland nicht allein.\u200b\n\nFast allein ist das Land aber mit der Möglichkeit, auf Autobahnen so schnell zu fahren, wie man will. In Europa hat nur ein anderer Ort kein Limit: die Isle of Man in der Irischen See. Autobahnen gibt es dort aber keine. Auch insgesamt gibt es nur wenige andere Länder ohne 

In [12]:
result["translation"]= result["translation"][result["translation"].index("{"):]
result["translation"] = result["translation"].replace("\n", "").rstrip()
if not result["translation"].endswith("}"):
    result["translation"]  = result["translation"] + "\"}]}"

In [181]:
result["translation"]

'{"data": [   {      "missing_entities": ["Bündnis Tempolimit jetzt!"],      "denser_summary": "57% der Deutschen unterstützen laut einer YouGov-Umfrage ein Tempolimit von 130 km/h auf Autobahnen. Sebastian Vettel ist Teil der Koalition Bündnis Tempolimit jetzt!, die sich für ein Geschwindigkeitslimit einsetzt. Verkehrsminister Volker Wissing ist gegen staatlich verordnete Geschwindigkeitsbegrenzungen und argumentiert, dass Autofahren Freiheit bedeutet. Die Debatte über Geschwindigkeitsbegrenzungen läuft seit 1953 und die mächtige deutsche Auto-Lobby sowie das Freiheitsargument spielen immer noch zentrale Rollen."   },   {      "missing_entities": ["Isle of Man"],      "denser_summary": "Nur die Isle of Man, ein Ort ohne Autobahnen, und einige andere Länder haben keine Geschwindigkeitsbegrenzung. Die deutsche Auto-Lobby und das Freiheitsargument sind immer noch zentral für die Debatte über Geschwindigkeitsbegrenzungen, die seit 1953 läuft. Eine YouGov-Umfrage ergab, dass 57% der Deutsc

In [273]:
import re
def fix_json_string(st):
    # finds all denser summarys, replaces quotation inside with &quot
    m = re.finditer(r'(?<=\"denser_summary\":)(.*?)(?=\})', st)

    new_string = ""
    idx = 0

    for i in list(m):
        ss, se = i.span(1)  # first and last index
        groups = i.group()  # complete string ins
        quotations = [m.start() for m in re.finditer('"', groups)]
        # Quotation inside dense summary?
        if(len(quotations)>2):
            new_string += st[idx:ss+quotations[1]]
            idx = ss+quotations[1]+1
            for quotindex in quotations[1:-1]:
                new_string += st[idx:ss+quotindex] + "“"
                idx = ss+quotindex+1
            new_string += st[idx:se]
            idx = quotations[-1]+1
        else:
            new_string += st[idx:ss] +  groups
        idx = se
    new_string += st[idx:]
    return new_string
fix_json_string(result["translation"])

{'data': [{'missing_entities': ['Bündnis Tempolimit jetzt!'],
   'denser_summary': '57% der Deutschen unterstützen laut einer YouGov-Umfrage ein Tempolimit von 130 km/h auf Autobahnen. Sebastian Vettel ist Teil der Koalition Bündnis Tempolimit jetzt!, die sich für ein Geschwindigkeitslimit einsetzt. Verkehrsminister Volker Wissing ist gegen staatlich verordnete Geschwindigkeitsbegrenzungen und argumentiert, dass Autofahren Freiheit bedeutet. Die Debatte über Geschwindigkeitsbegrenzungen läuft seit 1953 und die mächtige deutsche Auto-Lobby sowie das Freiheitsargument spielen immer noch zentrale Rollen.'},
  {'missing_entities': ['Isle of Man'],
   'denser_summary': 'Nur die Isle of Man, ein Ort ohne Autobahnen, und einige andere Länder haben keine Geschwindigkeitsbegrenzung. Die deutsche Auto-Lobby und das Freiheitsargument sind immer noch zentral für die Debatte über Geschwindigkeitsbegrenzungen, die seit 1953 läuft. Eine YouGov-Umfrage ergab, dass 57% der Deutschen ein Tempolimit von 

In [13]:
import json
jsoned =json.loads(result["translation"])
jsoned

{'data': [{'missing_entities': ['Bündnis Tempolimit jetzt!',
    'Verkehrsminister Volker Wissing',
    'Isle of Man'],
   'denser_summary': "57% der Deutschen unterstützen laut einer YouGov-Umfrage ein Tempolimit von 130 km/h auf Autobahnen. Sebastian Vettel ist Mitglied des Bündnisses Tempolimit jetzt!, das sich für das Limit einsetzt. Verkehrsminister Volker Wissing lehnt staatliche Beschränkungen im Straßenverkehr ab und argumentiert mit 'Fahren bedeutet Freiheit'. Deutschland ist eines der wenigen Länder ohne Höchstgeschwindigkeitsbegrenzung, zusammen mit der Isle of Man, die keine Autobahnen hat. Die Debatte über das Tempolimit dauert seit 1953 an und wird trotz der Umwelt- und Sicherheitsbedenken weiterhin durch die mächtige deutsche Auto-Lobby und 'Freiheits'-Argumente befeuert."},
  {'missing_entities': ['1953',
    'Konrad Adenauer',
    '80 km/h außerhalb von Ortschaften'],
   'denser_summary': 'Nach einer großen Deregulierung der Geschwindigkeitsbegrenzungen im Jahr 1953 un

In [ ]:
cleaned = []
for (i, element) in enumerate(jsoned['data']):
    print(element)
    missing = element['missing_entities']
    if(isinstance(missing, str)):
        element['missing_entities'] = [missing]
    cleaned.append(element)

{'missing_entities': 'Bündnis Tempolimit jetzt!, Sebastian Vettel', 'denser_summary': 'Laut einer YouGov-Umfrage unterstützen 57% der Deutschen ein Tempolimit von 130 km/h auf Autobahnen. Das Bündnis Tempolimit jetzt!, zu dem auch Sebastian Vettel gehört, setzt sich für eine Geschwindigkeitsbegrenzung ein. Verkehrsminister Volker Wissing von der FDP ist jedoch gegen staatlich verordnete Geschwindigkeitsbeschränkungen. Deutschland ist eines der wenigen Länder in Europa ohne Höchstgeschwindigkeitsbegrenzung, aber die Debatte dauert seit Jahrzehnten an, wobei Sicherheitsbedenken und der Klimawandel wichtige Faktoren sind.'}
{'missing_entities': 'Konrad Adenauer', 'denser_summary': '1953, unter Kanzler Konrad Adenauer, erfolgte eine umfassende Deregulierung der Geschwindigkeitsbegrenzungen, die es den Fahrern ermöglichte, so schnell zu fahren, wie sie wollten. Die Einführung einer Geschwindigkeitsbegrenzung von 100 km/h auf Landstraßen im Jahr 1972 stieß auf Widerstand, ebenso wie die vorü

In [ ]:
cleaned

[{'missing_entities': ['Bündnis Tempolimit jetzt!, Sebastian Vettel'],
  'denser_summary': 'Laut einer YouGov-Umfrage unterstützen 57% der Deutschen ein Tempolimit von 130 km/h auf Autobahnen. Das Bündnis Tempolimit jetzt!, zu dem auch Sebastian Vettel gehört, setzt sich für eine Geschwindigkeitsbegrenzung ein. Verkehrsminister Volker Wissing von der FDP ist jedoch gegen staatlich verordnete Geschwindigkeitsbeschränkungen. Deutschland ist eines der wenigen Länder in Europa ohne Höchstgeschwindigkeitsbegrenzung, aber die Debatte dauert seit Jahrzehnten an, wobei Sicherheitsbedenken und der Klimawandel wichtige Faktoren sind.'},
 {'missing_entities': ['Konrad Adenauer'],
  'denser_summary': '1953, unter Kanzler Konrad Adenauer, erfolgte eine umfassende Deregulierung der Geschwindigkeitsbegrenzungen, die es den Fahrern ermöglichte, so schnell zu fahren, wie sie wollten. Die Einführung einer Geschwindigkeitsbegrenzung von 100 km/h auf Landstraßen im Jahr 1972 stieß auf Widerstand, ebenso w

In [191]:
from __future__ import annotations

import tiktoken

MODELS_2_TOKEN_LIMITS = {
    "gpt-35-turbo": 4000,
    "gpt-3.5-turbo": 4000,
    "gpt-35-turbo-16k": 16000,
    "gpt-3.5-turbo-16k": 16000,
    "gpt-4": 8100,
    "gpt-4-32k": 32000
}

AOAI_2_OAI = {
    "gpt-35-turbo": "gpt-3.5-turbo",
    "gpt-35-turbo-16k": "gpt-3.5-turbo-16k"
}


def get_token_limit(model_id: str) -> int:
    if model_id not in MODELS_2_TOKEN_LIMITS:
        raise ValueError("Expected model gpt-35-turbo and above")
    return MODELS_2_TOKEN_LIMITS[model_id]


def num_tokens_from_messages(messages: 'list[dict[str, str]]', model: str) -> int:
    num_tokens = 0
    for conversation in messages:
        if("user" in conversation and conversation["user"]):
            userMsg = conversation["user"]
            num_tokens += num_tokens_from_message(message= userMsg, model=model)
        if("bot" in conversation and conversation["bot"]):
            aiMsg = conversation["bot"]
            num_tokens += num_tokens_from_message(message= aiMsg, model=model)
    return num_tokens
           
def num_tokens_from_message(message: str, model: str, token_per_message: int = 3) -> int:
    """
    Calculate the number of tokens required to encode a message.
    Args:
        message (str): The message to encode
        model (str): The name of the model to use for encoding.
        token_per_message (number): offset per message
    Returns:
        int: The total number of tokens required to encode the message.
    Example:
        message = {'role': 'user', 'content': 'Hello, how are you?'}
        model = 'gpt-3.5-turbo'
        num_tokens_from_messages(message, model)
        output: 11
    """
    encoding = tiktoken.encoding_for_model(get_oai_chatmodel_tiktok(model))
    num_tokens = token_per_message  # For "role" and "content" keys
    num_tokens += len(encoding.encode(message))
    return num_tokens


def get_oai_chatmodel_tiktok(aoaimodel: str) -> str:
    message = "Expected Azure OpenAI ChatGPT model name"
    if aoaimodel == "" or aoaimodel is None:
        raise ValueError(message)
    if aoaimodel not in AOAI_2_OAI and aoaimodel not in MODELS_2_TOKEN_LIMITS:
        raise ValueError(message)
    return AOAI_2_OAI.get(aoaimodel) or aoaimodel

In [192]:
sumprompt = getSummarizationPrompt()
translateprompt = getTranslationPrompt()
num_tokens_from_message(sumprompt.format(text=text), "gpt-35-turbo", 0) + num_tokens_from_message(translateprompt.format(language=language, sum= result["sum"]), "gpt-35-turbo", 0) #prompt tokens

1089

In [193]:
translateprompt.format(language=language, sum= result["sum"])

'\nÜbersetze das folgende JSON in deutsch. Beinhalte die Formatierung als RFC8259 JSON bei.\n\nJSON: {\n"data": [\n   {\n      "missing_entities": "Anna, a student"\n      "denser_summary": "Anna, a 16-year-old student, wakes up at 7am to a ringing alarm clock. She has breakfast, makes her bed, and catches the school bus. After school, she does homework and spends time with friends before dinner. She watches TV before going to bed at 10pm."\n   },\n   {\n      "missing_entities": "toilet, face washing"\n      "denser_summary": "Anna wakes up to an alarm clock, uses the toilet, and washes her face before showering. She gets dressed and has breakfast before making her bed and heading to school."\n   },\n   {\n      "missing_entities": "cold water, toothbrushing"\n      "denser_summary": "Anna showers with warm and cold water to wake up, brushes her teeth after getting dressed, and catches the bus to school after breakfast. She does homework and hangs out with friends in the afternoon."\n

In [194]:
num_tokens_from_message(result["translation"], "gpt-35-turbo", 0) + num_tokens_from_message(result["sum"], "gpt-35-turbo",0) #result tokens


815

In [195]:
cb.prompt_tokens, cb.completion_tokens

(1105, 815)